In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
#gpu configuration
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
!nvidia-smi

In [ ]:
#import the libraries as shown below

from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob

In [ ]:
import numpy as np
import pandas as pd

In [11]:
#Training images and csv
train_images=glob('../input/plant-pathology-2021-fgvc8/train_images/*')
train_csv=pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
print(len(train_images))

18632


In [ ]:
submission_csv=pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
print(submission_csv.head(n=3))

In [ ]:
print(train_images[:10])
print(train_csv.head(n=6))

In [ ]:
#Convert target to numeric data
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder(dtype=int)
train_csv['labels']=oe.fit_transform(train_csv[['labels']])

print(type(train_csv))
print(train_csv)

In [ ]:
#Extract the label column
train_labels=train_csv.labels

In [ ]:
#Load half of thr training dataset due to memory issues
train_copy=train_images[:500]
print(len(train_copy))
labels_copy=train_labels[:500]
print(len(labels_copy))

In [ ]:
#Check for imbalancednes among the training labels
from collections import Counter
print(Counter(labels_copy))

In [ ]:
#Number of unique diseases
out_shape=labels_copy.nunique()
print(out_shape)

The dataset is very imblanced.

In [ ]:
y=np.array(train_labels)
print(type(y))

In [ ]:
#Test images loading
test_images=glob('../input/plant-pathology-2021-fgvc8/test_images/*')
print(len(test_images))

# Visualize the images at random

In [ ]:
import matplotlib.pyplot as plt
from skimage import io 
import cv2 

In [ ]:
flower_files=train_csv['image']

#Visualize the images randomly from the dataset
n=np.random.randint(0,18692,1)[0]
img=io.imread('../input/plant-pathology-2021-fgvc8/train_images/'+flower_files[n])
rgb_img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(3,3))
plt.imshow(rgb_img)
print(img.shape)

# Convert images to array

In [ ]:
#Convert Images to array
train = []
test=[]

for myFile in train_images:
    im=cv2.imread(myFile, cv2.COLOR_BGR2RGB)
    img=cv2.resize(im,(224,224))
    image=np.asarray(img,dtype=np.float32)
    train.append(image)

for myFile in test_images:
    im=cv2.imread(myFile,cv2.COLOR_BGR2RGB)
    img=cv2.resize(im,(224,224))
    image=np.asarray(img,dtype=np.float32)
    test.append(image)

In [ ]:
#Save as  h5py file
import h5py
with h5py.File('train.h5','w') as h5f:
    h5f.create_dataset('arrays',data=np.asarray(train))

with h5py.File('test.h5','w') as h5f:
    h5f.create_dataset('arrays',data=np.asarray(test))

In [ ]:
#Define X_train,y_train and test
X_train=np.asarray(train,dtype=np.float32)
X= X_train.reshape(-1,224,224,3)/255
print(X_train.shape,",",X_train.dtype)
y= np.array(labels_copy)
print(y_train.dtype,":",y_train.shape)
test=np.asarray(test,dtype=np.float32).reshape(-1,224,224,3)/255



# Loading data using ImagedataGenerator

In [ ]:
#This will add .png to every observation in the id column.Useful when not present.
def append_ext(fn):
    return fn+".png"def append_ext(fn):
    return fn+".png"

traindf=pd.read_csv(“./trainLabels.csv”,dtype=str)

testdf=pd.read_csv("./sampleSubmission.csv",dtype=str)traindf["id"]=traindf["id"].apply(append_ext)

testdf["id"]=testdf["id"].apply(append_ext)

You would have noticed, I appended “.png” to all the filenames in the “id” column of the dataframe to convert the file ids to actual filenames(depending upon the dataset you might want to handle this accordingly), previously that was handled automatically by “has_ext” attribute which is now deprecated for various reasons.

In [ ]:
# Use the Image Data Generator to import the images from the dataset
train_datagen=ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,
                                 horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
#Generate training data
train_generator=train_datagen.flow_from_dataframe(dataframe=train_csv[:5000],directory='../input/plant-pathology-2021-fgvc8/train_images',
                                            x_col='image',y_col='labels',subset='training',
                                            batch_size=32,seed=42,shuffle=False,
                                            class_mode='categorical',target_size=(224,224))

Found 5000 validated image filenames belonging to 12 classes.


In [ ]:
print(train_generator.class_indices)

In [ ]:
#Generate validation data
valid_generator=train_datagen.flow_from_dataframe(dataframe=train_csv[5000:6500],directory='../input/plant-pathology-2021-fgvc8/train_images',
                                            x_col='image',y_col='labels',subset='training',
                                            batch_size=32,seed=42,shuffle=False,
                                            class_mode='categorical',target_size=(224,224))

In [ ]:
#Generate test data
test_generator=test_datagen.flow_from_dataframe(dataframe=submission_csv,directory='../input/plant-pathology-2021-fgvc8/test_images',
                                          target_size=(224, 224),batch_size=32,x_col='image',
                                          class_mode=None)

# ResNet50 transfer learning

In [ ]:
#re-size all the images to this
IMAGE_SIZE=[224, 224]

#ResNet50
resnet=ResNet50(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

#don't train existing weights
for layer in resnet.layers:
    layer.trainable=False

#Faltten the layers
x=Flatten()(resnet.output)

#Final dense output layer
prediction=Dense(12,activation='softmax')(x)

#create a model object
model=Model(inputs=resnet.input,outputs=prediction)

#view the structure of the model
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
from keras.optimizers import Adam
adam=Adam(learning_rate=3e-4)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [ ]:
#Fit the model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

callback_list=[EarlyStopping(monitor='accuracy',patience=5)]

history=model.fit_generator(generator=train_generator,steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,
                    callbacks=callback_list,epochs=20)

In [ ]:
# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#Save the model
model.save('resnet50.h5')

# InceptionV3 modeling

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

#re-size all the images to this
IMAGE_SIZE=[224, 224]

#InceptionV3
inc_v3=InceptionV3(input_shape=IMAGE_SIZE + [3],pooling='avg',weights='imagenet',
                   include_top=False)

#don't train existing weights
for layer in inc_v3.layers:
    layer.trainable=False

#Faltten the layers
x=Flatten()(inc_v3.output)

#Final dense output layer
prediction=Dense(12,activation='softmax')(x)

#create a model object
model=Model(inputs=inc_v3.input,outputs=prediction)

#view the structure of the model
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
from keras.optimizers import Adam
adam=Adam(learning_rate=3e-4)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [ ]:
#Fit the model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

callback_list=[EarlyStopping(monitor='accuracy',patience=5)]

history=model.fit_generator(generator=train_generator,steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,
                    callbacks=callback_list,epochs=20)

In [ ]:
import matplotlib.pyplot as plt
# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#Save the model
model.save('inception_v31 .h5')

In [7]:
# load model
from tensorflow.keras.models import load_model
model=load_model('../input/inceptionv3/inception_v31 .h5')

# Predict

In [18]:
#Predict using predict_generator
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

labels=(train_generator.class_indices)
labels=dict((v,k) for k,v in labels.items())
predictions=[labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
results=pd.DataFrame({'image':filenames,
                      'labels':predictions})
print(results.head)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


1/1 [==============================] - 1s 772ms/step
<bound method NDFrame.head of                   image              labels
0  85f8cb619c66b863.jpg  frog_eye_leaf_spot
1  ad8770db05586b59.jpg             complex
2  c7b03e718489f3ca.jpg  frog_eye_leaf_spot>


In [19]:
results.to_csv('sample_submission.csv',index=False)

In [17]:
# remove the file
import os
os.remove('./sample_submission.csv')
os.remove('./results.csv')